In [1]:
import RAP_MASTER_LIB
#MY LIBRARIES

from rap_scrpr import *
from rap_db import *
from rap_clean import*

estconn = pg2.connect(database='rap_songs', user='keenan', host='localhost', password='keenan')
#create_music_tables(estconn)

import RAP_MASTER_LIB
import re
from bs4 import BeautifulSoup
import urllib
import time
import json
import psycopg2 as pg2
import psycopg2.extras

from rap_scrpr import *
from rap_db import *
from rap_clean import*
#nltk.download()
from os import listdir
from os.path import isfile, join

estconn = pg2.connect(database='rap_songs', user='keenan', host='localhost', password='keenan')
#create_music_tables(estconn)

In [2]:
#for show
#scrape artist
def scrape_multi_artists(conn, artist_list):
    ret_list = []
    for art in artist_list:
        page = pull_link(conn, art)
        scraped_songs = full_artist_scrape(page, art)
        art_file = art.replace(' ', '_').lower()
        #back up scrape after clean
        with open('json_lyrics/'+art_file+'_raw.json', 'w') as outfile:
            json.dump(scraped_songs, outfile)
        print(art_file+'_raw.json made!')
        ret_list.append(art_file+'_raw.json')
    return ret_list

#load them into db
def bulk_load(conn, new_eds = []):
    if not new_eds:
        new_eds = [f for f in listdir('json_lyrics/') if isfile(join('json_lyrics/', f))]
    for new in new_eds:
        songs = json.load(open('json_lyrics/'+new))
        art_name = new.replace("_raw.json","")
        add_base(conn, art_name)
        for name, works in songs.items():
            if 'raw_song_' not in name:
                add_songs(conn, art_name, name, works)
        print(new+" added!")

#MAY GET RID OF THIS OR USE PERCISE PULL FOR PERFORMANCE
        
#pull from db and make song objects
def construct_artists(art_list = [''], alb_list = [''], sng_list = ['']):
    #pull from db
    db_records = adv_pull(estconn, art_list, alb_list, sng_list)
    
    artists = []
    #make song objs
    for art, albs in db_records.items():
        
        album_objs = []
        for alb_name, sngs in albs.items():
            song_objs = []
            for sng_name, lyrc in sngs.items():
                song_obj = song(lyrc, sng_name, art)
                song_obj.assign_extras()
                song_obj.remove_and_reass()
                song_obj.create_song_as_seg()
                song_objs.append(song_obj)
            album_obj = album(art, alb_name, song_objs)
            #extra verse cleaning
            for v in album_obj.verses:
                v.run_all_split()
            album_objs.append(album_obj)

        artist_obj = artist(art, album_objs)
        artists.append(artist_obj)
    return artists

In [3]:
#boolean tells it if its one of two style pages, in final product scrape an artist
#artists_to_scrape =  ['Buckwild']
#HAVE TO RESCRAPE, CLEANING PROCESS TOOK OUT VITAL INFO
#artist_to_scrape = {'del':'http://www.ohhla.com/YFA_del.html'}
#create_music_tables(estconn, bypass = True)
#added = scrape_multi_artists(estconn, artists_to_scrape)
#now safely stored in json file filled folder
#create_music_tables(estconn, bypass = True)
#now load them into db
#bulk_load(estconn, added)

In [4]:
ar = construct_artists(['Chance The Rapper', 'Odd Future'], ['10 Day (Mixtape)', 'Acid Rap'])

In [5]:
ar[0].albums

[<rap_clean.album at 0x1f84c1dd9b0>, <rap_clean.album at 0x1f84c2aeeb8>]

what you did

so much hard to tell tbh
made artist, album classes, finished clean library and added all verse cleaning,
made an upgraded way to pull from db,
set up first function to feed artists and spit out objects, smart too
touched up typos, cleaned clean library
also made easier way to scrape
soooo much shit
probably need cleaning before final publsih
and consolidated master library

next is viz and modeling forsure


In [7]:
estconn.close()